In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from PIL import Image
import os
import copy
import pandas as pd
from sklearn.metrics import f1_score

labels_csv_path = 'Training.csv'
df = pd.read_csv(labels_csv_path)

image_files = df['Image'].values
labels = df['Label'].values

# Split the data into training and validation sets
train_files, val_files, train_labels, val_labels = train_test_split(image_files, labels, test_size=0.2, random_state=42)

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, directory, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.directory = directory
        self.transform = transform
        self.label_mapping = {
            'Normal': 0,
            'Mitosis': 1
        }

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = self.image_paths[idx]
        label_text = self.labels[idx]
        label = self.label_mapping[label_text]
        label_tensor = torch.tensor(label, dtype=torch.float32)
        
        img_path = f"{self.directory}/{img_name}"
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        
        return image, label_tensor

transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Create datasets
train_dataset = CustomDataset(train_files, train_labels, 'Training Image', transform=transform)
val_dataset = CustomDataset(val_files, val_labels, 'Training Image', transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)

# Model Architecture
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        # Adjust the input features to match the actual size
        self.fc1 = nn.Linear(128 * 18 * 18, 512)  # This line might need adjustment
        self.fc2 = nn.Linear(512, 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 128 * 18 * 18)  # This line might need adjustment
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Model, Loss, Optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNNModel().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

labels_unique, counts = np.unique(train_labels, return_counts=True)
class_weights = compute_class_weight('balanced', classes=labels_unique, y=train_labels)
pos_weight = torch.tensor([class_weights[1] / class_weights[0]], dtype=torch.float).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)


# Early Stopping
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

# Training Function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=25, patience=7):
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0
        train_preds, train_targets = [], []
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            train_preds.extend(torch.sigmoid(outputs).detach().cpu().numpy().round())
            train_targets.extend(labels.detach().cpu().numpy())

        train_f1 = f1_score(train_targets, train_preds)

        val_loss = 0.0
        val_preds, val_targets = [], []
        model.eval()
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_preds.extend(torch.sigmoid(outputs).detach().cpu().numpy().round())
                val_targets.extend(labels.detach().cpu().numpy())

        val_f1 = f1_score(val_targets, val_preds)

        print(f'Epoch {epoch+1}, Train Loss: {total_loss/len(train_loader):.4f}, Train F1: {train_f1:.4f}, Val Loss: {val_loss/len(val_loader):.4f}, Val F1: {val_f1:.4f}')
        
        early_stopping(val_loss/len(val_loader), model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break

    model.load_state_dict(torch.load('checkpoint.pt'))

# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20)

Epoch 1, Train Loss: 1.8129, Train F1: 0.6032, Val Loss: 1.0913, Val F1: 0.0000
Validation loss decreased (inf --> 1.091287).  Saving model ...
Epoch 2, Train Loss: 0.8258, Train F1: 0.4390, Val Loss: 0.7956, Val F1: 0.7619
Validation loss decreased (1.091287 --> 0.795620).  Saving model ...
Epoch 3, Train Loss: 0.6942, Train F1: 0.8108, Val Loss: 0.9110, Val F1: 0.6667
EarlyStopping counter: 1 out of 7
Epoch 4, Train Loss: 0.5441, Train F1: 0.8235, Val Loss: 1.2521, Val F1: 0.6667
EarlyStopping counter: 2 out of 7
Epoch 5, Train Loss: 0.5999, Train F1: 0.8116, Val Loss: 2.4588, Val F1: 0.4286
EarlyStopping counter: 3 out of 7
Epoch 6, Train Loss: 0.6521, Train F1: 0.7636, Val Loss: 1.2326, Val F1: 0.7368
EarlyStopping counter: 4 out of 7
Epoch 7, Train Loss: 0.7522, Train F1: 0.8333, Val Loss: 1.3873, Val F1: 0.6667
EarlyStopping counter: 5 out of 7
Epoch 8, Train Loss: 0.6590, Train F1: 0.7937, Val Loss: 2.0639, Val F1: 0.4615
EarlyStopping counter: 6 out of 7
Epoch 9, Train Loss: 0.

In [6]:
class TestDataset(Dataset):
    def __init__(self, image_paths, directory, transform=None):
        self.image_paths = image_paths
        self.directory = directory
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = self.image_paths[idx]
        img_path = f"{self.directory}/{img_name}"
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

test_df = pd.read_csv('Testing.csv')
test_image_files = test_df['Image'].values

test_transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

test_dataset = TestDataset(test_image_files, 'Testing Image', transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False)

In [15]:
def predict(model, test_loader, device):
    model.eval()  # Set the model to evaluation mode
    predictions = []
    with torch.no_grad():  # No need to track gradients for predictions
        for inputs in test_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = torch.sigmoid(outputs).data > 0.5  # Applying threshold to convert to binary predictions
            predictions.extend(preds.long().cpu().numpy())
    return np.array(predictions).flatten()

# Make predictions
test_predictions = predict(model, test_loader, device)

print("Test Predictions:", test_predictions)

Test Predictions: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight

# Define transforms with augmentation
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.RandomPerspective(distortion_scale=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


# Create datasets and loaders
train_dataset = CustomDataset(train_files, train_labels, 'Training Image', transform=transform)
val_dataset = CustomDataset(val_files, val_labels, 'Training Image', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)

# Model architecture
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 18 * 18, 512)
        self.fc2 = nn.Linear(512, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Assuming binary classification
labels_unique = np.unique(train_labels)
class_weights = compute_class_weight('balanced', classes=labels_unique, y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor[1])

def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    best_val_loss = float('inf')
    patience = 5  # Number of epochs to wait for improvement before stopping
    patience_counter = 0  # Counter for patience
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5, min_lr=1e-6, verbose=True)
    
    for epoch in range(num_epochs):
        model.train()
        train_losses = []
        train_preds, train_targets = [], []
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_losses.append(loss.item())
            
            preds = torch.sigmoid(outputs).data > 0.5
            train_preds.extend(preds.cpu().numpy())
            train_targets.extend(labels.cpu().numpy())

        train_acc = accuracy_score(train_targets, train_preds)
        train_f1 = f1_score(train_targets, train_preds)

        model.eval()
        val_losses = []
        val_preds, val_targets = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_losses.append(loss.item())
                
                preds = torch.sigmoid(outputs).data > 0.5
                val_preds.extend(preds.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())
        
        val_acc = accuracy_score(val_targets, val_preds)
        val_f1 = f1_score(val_targets, val_preds)
        
        # Early Stopping Check
        if np.mean(val_losses) < best_val_loss:
            best_val_loss = np.mean(val_losses)
            patience_counter = 0  # Reset counter
            # Save the best model
            torch.save(model.state_dict(), "best_model3.pth")
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print("Early stopping triggered")
            break
        
        scheduler.step(np.mean(val_losses))
        current_lr = scheduler.get_last_lr()[0]
        print(f'Current learning rate: {current_lr}')
        
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {np.mean(train_losses):.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Loss: {np.mean(val_losses):.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')

model3 = train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=20)

c:\Users\Fatima Azfar\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Current learning rate: 0.001
Epoch 1/20, Train Loss: 20.2735, Train Acc: 0.5192, Train F1: 0.6667, Val Loss: 1.6647, Val Acc: 0.3571, Val F1: 0.0000
Current learning rate: 0.001
Epoch 2/20, Train Loss: 10.7657, Train Acc: 0.4231, Train F1: 0.0000, Val Loss: 5.7535, Val Acc: 0.3571, Val F1: 0.0000
Current learning rate: 0.001
Epoch 3/20, Train Loss: 1.5588, Train Acc: 0.5769, Train F1: 0.5769, Val Loss: 1.2053, Val Acc: 0.6429, Val F1: 0.7826
Current learning rate: 0.001
Epoch 4/20, Train Loss: 2.3730, Train Acc: 0.5769, Train F1: 0.7317, Val Loss: 1.5536, Val Acc: 0.6429, Val F1: 0.7826
Current learning rate: 0.001
Epoch 5/20, Train Loss: 2.0426, Train Acc: 0.5769, Train F1: 0.7317, Val Loss: 2.2239, Val Acc: 0.6429, Val F1: 0.7826
Current learning rate: 0.001
Epoch 6/20, Train Loss: 0.7220, Train Acc: 0.6923, Train F1: 0.7838, Val Loss: 0.5038, Val Acc: 0.5714, Val F1: 0.5000
Current learning rate: 0.001
Epoch 7/20, Train Loss: 1.1605, Train Acc: 0.5577, Train F1: 0.4889, Val Loss: 1.

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, f1_score
import numpy as np

labels_csv_path = 'Training.csv'
df = pd.read_csv(labels_csv_path)

image_files = df['Image'].values
labels = df['Label'].values

# Custom Dataset
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, directory, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.directory = directory
        self.transform = transform
        self.label_mapping = {
            'Normal': 0,
            'Mitosis': 1
        }

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = self.image_paths[idx]
        label_text = self.labels[idx]
        label = self.label_mapping[label_text]
        label_tensor = torch.tensor(label, dtype=torch.float32)
        
        img_path = f"{self.directory}/{img_name}"
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)
        
        return image, label_tensor

# Data augmentation and normalization for training
# Just normalization for validation
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.RandomPerspective(distortion_scale=0.05),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Splitting dataset
train_files, val_files, train_labels, val_labels = train_test_split(image_files, labels, test_size=0.2, random_state=42)

# Dataset and DataLoader
train_dataset = CustomDataset(train_files, train_labels, 'Training Image', transform=transform)
val_dataset = CustomDataset(val_files, val_labels, 'Training Image', transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Model
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1)  # Change the final layer

model = model.to(device)

labels_unique = np.unique(train_labels)
class_weights = compute_class_weight('balanced', classes=labels_unique, y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor[1])

optimizer = optim.Adam(model.parameters(), lr=0.001)

# Function to calculate accuracy and F1 score
def calculate_metrics(outputs, labels):
    preds = torch.sigmoid(outputs).data > 0.5
    preds = preds.to(torch.float32)
    acc = accuracy_score(labels.cpu(), preds.cpu())
    f1 = f1_score(labels.cpu(), preds.cpu())
    return acc, f1

# Training and Validation Loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss, running_acc, running_f1 = 0.0, 0.0, 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        acc, f1 = calculate_metrics(outputs, labels)
        running_acc += acc
        running_f1 += f1

    # Validation
    model.eval()
    val_loss, val_acc, val_f1 = 0.0, 0.0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device).float().view(-1, 1)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            acc, f1 = calculate_metrics(outputs, labels)
            val_acc += acc
            val_f1 += f1

    print(f'Epoch {epoch+1}/{num_epochs}, '
          f'Train Loss: {running_loss/len(train_loader):.4f}, Train Acc: {running_acc/len(train_loader):.4f}, Train F1: {running_f1/len(train_loader):.4f}, '
          f'Val Loss: {val_loss/len(val_loader):.4f}, Val Acc: {val_acc/len(val_loader):.4f}, Val F1: {val_f1/len(val_loader):.4f}')

print('Finished Training')

c:\Users\Fatima Azfar\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Fatima Azfar\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Train Loss: 0.7561, Train Acc: 0.6031, Train F1: 0.4167, Val Loss: 2.3239, Val Acc: 0.5714, Val F1: 0.7000
Epoch 2/10, Train Loss: 1.0367, Train Acc: 0.6312, Train F1: 0.7338, Val Loss: 13.2506, Val Acc: 0.4286, Val F1: 0.4286
Epoch 3/10, Train Loss: 0.5122, Train Acc: 0.7500, Train F1: 0.7900, Val Loss: 22.1274, Val Acc: 0.5000, Val F1: 0.5333
Epoch 4/10, Train Loss: 0.7421, Train Acc: 0.7000, Train F1: 0.7704, Val Loss: 35.3358, Val Acc: 0.3571, Val F1: 0.0000
Epoch 5/10, Train Loss: 0.5638, Train Acc: 0.7656, Train F1: 0.8177, Val Loss: 18.0069, Val Acc: 0.3571, Val F1: 0.5263
Epoch 6/10, Train Loss: 0.4792, Train Acc: 0.8406, Train F1: 0.8801, Val Loss: 17.1516, Val Acc: 0.6429, Val F1: 0.7826
Epoch 7/10, Train Loss: 0.4676, Train Acc: 0.7625, Train F1: 0.8024, Val Loss: 3.8061, Val Acc: 0.6429, Val F1: 0.7826
Epoch 8/10, Train Loss: 0.4890, Train Acc: 0.7812, Train F1: 0.8250, Val Loss: 2.2620, Val Acc: 0.6429, Val F1: 0.7826
Epoch 9/10, Train Loss: 0.4345, Train Acc: 

In [28]:
def predict(model, data_loader):
    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs in data_loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            preds = torch.sigmoid(outputs).data > 0.5
            predictions.extend(preds.long().cpu().numpy())
    return np.array(predictions).flatten()

# Make predictions
test_predictions = predict(model, test_loader)

In [29]:
label_mapping_inverse = {0: 'Normal', 1: 'Mitosis'}
textual_predictions = [label_mapping_inverse[pred] for pred in test_predictions]
test_df['Label'] = textual_predictions
test_df.to_csv('Test_Predictions_11.csv', index=False)

In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
from PIL import Image

# Load the pre-trained DenseNet model
model = models.densenet121(pretrained=True)

# Modify the classifier for binary classification
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 1)  # Output layer for binary classification

# Use GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the criterion with class weights
class_weights = compute_class_weight('balanced', classes=np.unique(train_labels), y=train_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights_tensor[1])

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training function adapted for DenseNet
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        train_preds, train_targets = [], []
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            
            train_preds += torch.round(torch.sigmoid(outputs)).squeeze().tolist()
            train_targets += labels.tolist()
        
        train_acc = accuracy_score(train_targets, train_preds)
        train_f1 = f1_score(train_targets, train_preds)

        model.eval()
        val_preds, val_targets = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                val_preds += torch.round(torch.sigmoid(outputs)).squeeze().tolist()
                val_targets += labels.tolist()
        
        val_acc = accuracy_score(val_targets, val_preds)
        val_f1 = f1_score(val_targets, val_preds)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')

train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10)

c:\Users\Fatima Azfar\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Fatima Azfar\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10, Loss: 0.5301, Train Acc: 0.6346, Train F1: 0.6122, Val Acc: 0.5000, Val F1: 0.5882
Epoch 2/10, Loss: 0.4551, Train Acc: 0.8077, Train F1: 0.8529, Val Acc: 0.5714, Val F1: 0.6667
Epoch 3/10, Loss: 0.4645, Train Acc: 0.8462, Train F1: 0.8788, Val Acc: 0.5000, Val F1: 0.5333
Epoch 4/10, Loss: 0.3608, Train Acc: 0.9038, Train F1: 0.9153, Val Acc: 0.5714, Val F1: 0.7000
Epoch 5/10, Loss: 0.4443, Train Acc: 0.9038, Train F1: 0.9180, Val Acc: 0.5714, Val F1: 0.7000
Epoch 6/10, Loss: 0.2304, Train Acc: 0.9231, Train F1: 0.9310, Val Acc: 0.5714, Val F1: 0.6667
Epoch 7/10, Loss: 0.1960, Train Acc: 0.9808, Train F1: 0.9836, Val Acc: 0.5000, Val F1: 0.5333
Epoch 8/10, Loss: 0.1365, Train Acc: 0.9423, Train F1: 0.9508, Val Acc: 0.5000, Val F1: 0.5882
Epoch 9/10, Loss: 0.1871, Train Acc: 0.9615, Train F1: 0.9677, Val Acc: 0.5000, Val F1: 0.4615
Epoch 10/10, Loss: 0.1167, Train Acc: 0.9808, Train F1: 0.9831, Val Acc: 0.5714, Val F1: 0.5714


# DenseNet121: Best Model so far

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import pandas as pd
from PIL import Image
import copy
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset

# Load the dataset
labels_csv_path = 'Training.csv'
df = pd.read_csv(labels_csv_path)
labels = df['Label'].values

# Encode labels
label_mapping = {'Normal': 0, 'Mitosis': 1}
encoded_labels = np.array([label_mapping[label] for label in labels])

image_files = df['Image'].values

# Split the dataset
train_files, val_files, train_labels, val_labels = train_test_split(image_files, encoded_labels, test_size=0.2, random_state=42)

# Dataset class with augmentation
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, directory, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.directory = directory
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_name = self.image_paths[idx]
        label = self.labels[idx]
        img_path = f"{self.directory}/{img_name}"
        image = Image.open(img_path).convert('RGB')
        
        if self.transform:
            image = self.transform(image)
        
        return image, torch.tensor(label, dtype=torch.float32)

# Define transforms with augmentation
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Create datasets and loaders
train_dataset = CustomDataset(train_files, train_labels, 'Training Image', transform=transform)
val_dataset = CustomDataset(val_files, val_labels, 'Training Image', transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=20, shuffle=False)

# Load the pre-trained DenseNet model
model = models.densenet121(pretrained=True)

# Modify the classifier for binary classification and add dropout
num_ftrs = model.classifier.in_features
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(num_ftrs, 1)
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Implement Focal Loss instead of weighted BCE
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        BCE_loss = nn.functional.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduction == 'mean':
            return torch.mean(F_loss)
        elif self.reduction == 'sum':
            return torch.sum(F_loss)
        else:
            return F_loss

criterion = FocalLoss().to(device)

# Optimizer with learning rate scheduling
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

def train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_f1 = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
                data_loader = train_loader
            else:
                model.eval()   # Set model to evaluate mode
                data_loader = val_loader

            running_loss = 0.0
            running_corrects = 0
            preds_list = []
            targets_list = []

            # Iterate over data.
            for inputs, labels in data_loader:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward
                # Track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    preds = torch.sigmoid(outputs).squeeze()
                    loss = criterion(outputs, labels.unsqueeze(1))

                    # Backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                preds_binary = torch.round(preds).detach().cpu().numpy()
                preds_list.extend(preds_binary)
                targets_list.extend(labels.cpu().numpy())

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / len(data_loader.dataset)
            epoch_acc = accuracy_score(targets_list, preds_list)
            epoch_f1 = f1_score(targets_list, preds_list)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f} F1: {epoch_f1:.4f}')

            # Deep copy the model
            if phase == 'val' and epoch_f1 > best_f1:
                best_f1 = epoch_f1
                best_model_wts = copy.deepcopy(model.state_dict())

    print('Best val F1: {:4f}'.format(best_f1))

    # Load best model weights
    model.load_state_dict(best_model_wts)
    return model

model = train_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs=10)

c:\Users\Fatima Azfar\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Fatima Azfar\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/10
----------
Train Loss: 0.2076 Acc: 0.6346 F1: 0.6667
Val Loss: 0.7244 Acc: 0.4286 F1: 0.5000
Epoch 2/10
----------
Train Loss: 0.2368 Acc: 0.7500 F1: 0.7937
Val Loss: 0.6034 Acc: 0.5000 F1: 0.5882
Epoch 3/10
----------
Train Loss: 0.1191 Acc: 0.8462 F1: 0.8621
Val Loss: 0.5396 Acc: 0.6429 F1: 0.7368
Epoch 4/10
----------
Train Loss: 0.1387 Acc: 0.8654 F1: 0.8814
Val Loss: 0.7812 Acc: 0.6429 F1: 0.7368
Epoch 5/10
----------
Train Loss: 0.1093 Acc: 0.8462 F1: 0.8621
Val Loss: 1.1037 Acc: 0.5714 F1: 0.6667
Epoch 6/10
----------
Train Loss: 0.1128 Acc: 0.7692 F1: 0.8000
Val Loss: 0.7155 Acc: 0.7143 F1: 0.8000
Epoch 7/10
----------
Train Loss: 0.0932 Acc: 0.7885 F1: 0.8070
Val Loss: 0.7220 Acc: 0.6429 F1: 0.7826
Epoch 8/10
----------
Train Loss: 0.0617 Acc: 0.9423 F1: 0.9474
Val Loss: 0.4487 Acc: 0.7143 F1: 0.8182
Epoch 9/10
----------
Train Loss: 0.0470 Acc: 0.9038 F1: 0.9180
Val Loss: 0.3547 Acc: 0.7857 F1: 0.8571
Epoch 10/10
----------
Train Loss: 0.0333 Acc: 0.9615 F1: 0.9677

In [7]:
model.eval()  # Set the model to evaluation mode

predictions = []
with torch.no_grad():
    for images in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs).squeeze()
        predicted_labels = torch.round(preds).cpu().numpy()  # Convert to binary labels
        predictions.extend(predicted_labels)

# Map numeric predictions back to label names
label_mapping_inverse = {0: 'Normal', 1: 'Mitosis'}
predicted_labels_text = [label_mapping_inverse[int(pred)] for pred in predictions]

# Update the DataFrame with predictions
test_df['Label'] = predicted_labels_text

# Save the updated DataFrame
test_df.to_csv('Test_Preds_11.csv', index=False)

In [8]:
# Save the entire model
torch.save(model, 'densenet_full_model.pth')